In [3]:
import pickle
import pandas as pd
import copy
import os
import numpy as np
# import torch
from omegaconf import OmegaConf
import ast
import sys
# from definitions import RESULTS_DIR, ROOT_DIR_INSIDE
import pickle
import pandas as pd
import copy
import os
import numpy as np
import ast
# from xgboost import XGBClassifier
from sklearn import metrics
# root_dir = ROOT_DIR_INSIDE
# config = OmegaConf.load(f"{sys.argv[1]}/config.yaml")
input_data_dir = '/Users/projects_data/project_MLPhenotypesMRIGWAS/ml_generated_data/23_03_30/predictiveModel/ad_imaging.h5ad'
fname = os.path.basename(input_data_dir).split('.')[0]
# data_dir = f"{RESULTS_DIR}/prediction_results/example"
all_predictions = {}

def get_obs_data_anndata(fold_name):
    import anndata as ad
    adata = ad.read("/Users/projects_data/project_MLPhenotypesMRIGWAS/ml_generated_data/23_03_30/predictiveModel/ad_imaging.h5ad")
    split_column = "subject_id-DIAGNOSIS-StratifiedGroupKFold_GroupShuffleSplit_42_42"
    train_data = adata[adata.obsm['train_val_test_split'][~(adata.obsm['train_val_test_split'][split_column] .isin ([fold_name]))].index]
    val_data = adata[adata.obsm['train_val_test_split'][adata.obsm['train_val_test_split'][split_column] .isin ([fold_name])].index]
    X_train = np.array(train_data.X, dtype=np.float32)
    X_val = np.array(val_data.X, dtype=np.float32)
    y_train = np.array(train_data.obs['DIAGNOSIS'], dtype=np.float32)
    y_val = np.array(val_data.obs['DIAGNOSIS'], dtype=np.float32)
    return val_data.obs

obs = pd.concat([get_obs_data_anndata('fold1'), get_obs_data_anndata('fold2'), get_obs_data_anndata('fold3')])

In [18]:
csv_file_location = "/Users/EssentialCodeBase/project_MLPhenotypesMRIGWAS/block1_BaselineClassification/training/23_03_30/results_data/prediction_results/example/apr5/IMAGING_ADPREDICTION_CLASSIFIER/basic_numerical$imaging__addiagnosisprediction_train100$feature$sample"
for model_name in ['LogisticRegression', 'skorchMLPClassifier', 'XGBClassifier']:
    leaderboard_df = pd.read_csv(f"{csv_file_location}/csv_files/leaderboard_df.csv")
    agg_leaderboard_df = pd.read_csv(f"{csv_file_location}/csv_files/agg_leaderboard_df.csv")
    leaderboard_df = leaderboard_df[leaderboard_df['model_name'].isin([model_name])]
    agg_leaderboard_df = agg_leaderboard_df[agg_leaderboard_df['model_name'].isin([model_name])]
    config_mapping = dict(zip(leaderboard_df['config_index'], leaderboard_df['config'].map(lambda x: ast.literal_eval(x))))
    config_index1 = agg_leaderboard_df.iloc[0]['config_index']
    main_config_grid = OmegaConf.load(f"/Users/EssentialCodeBase/BaselineClassification/PredictiveModeling/src_basic/optuna_hyperparameters_with_index.json")
    classifier_params = main_config_grid['IMC'][model_name]['hp0']
    classifier_params.update(config_mapping[config_index1])
    dfg = classifier_params
    for key, val in dfg.items():
        try:
            dfg[key] = round(val, 2)
        except:
            pass
    print (dfg)

{'type': 'optuna', 'num_trials': 20, 'multi_class': 'multinomial', 'random_state': 42, 'FeatureSelection': 'IDENTITY', 'solver': 'saga', 'max_iter': 100, 'C': 275.25, 'penalty': 'l1'}
{'type': 'optuna', 'num_trials': 100, 'hidden_layer_sizes': [64, 32], 'max_epochs': 500, 'FeatureSelection': 'LASSO', 'optimizer': 'adamw', 'early_stopping': 1, 'random_state': 42, 'layers': 4, 'layer_size': [1, 512], 'learning_rate': [1e-05, 0.01], 'dropout': [0.0, 0.5], 'weight_decay': [1e-06, 0.001], 'layer_size0': 135, 'layer_size1': 275, 'layer_size2': 141, 'layer_size3': 11, 'module__dropout': 0.17, 'lr': 0.0, 'optimizer__weight_decay': 0.0, 'lasso_c': 0.06}
{'type': 'optuna', 'num_trials': 200, 'objective': 'multi:softprob', 'seed': 42, 'nthread': 4, 'silent': 1, 'n_estimators': 15, 'booster': 'gbtree', 'learning_rate': 0.09, 'reg_lambda': 1.89, 'gamma': 0.0, 'reg_alpha': 0.02, 'max_depth': 4, 'min_child_weight': 0.01, 'subsample': 0.4, 'colsample_bytree': 0.17, 'tree_method': 'gpu_hist', 'predicto

In [2]:
from pathlib import Path
RESULTS_DIR = Path("/Users/EssentialCodeBase/project_MLPhenotypesMRIGWAS/block1_BaselineClassification/training/23_03_30/results_data")

In [3]:
for model_name in ["LogisticRegression", "skorchMLPClassifier", "XGBClassifier"]:
    with open(RESULTS_DIR / f'predictiveAnalysis/execute_ensemble_models/ensemble_data/{fname}_{model_name}_best_model_prediction.pkl', 'rb') as t:
        all_predictions.update({f"{fname}#{i}":j for i,j in pickle.load(t).items()})

In [4]:
model_list = list(all_predictions)
model_list

from collections import defaultdict
model_list = list(all_predictions)
ll_loss_df = defaultdict(list)
full_dataframe_list = []
for e, model_name in enumerate(model_list):
    data_list = []
    for e, fold_name in enumerate(sorted(list(all_predictions[model_name]))):
        logloss_value = metrics.log_loss(all_predictions[model_name][fold_name]['y_val'], all_predictions[model_name][fold_name]['y_pred_val'])
        auc_value = metrics.roc_auc_score(all_predictions[model_name][fold_name]['y_val'], all_predictions[model_name][fold_name]['y_pred_val'][:, 1] , multi_class='ovo' )
        ll_loss_df['fold'].append(fold_name)
        ll_loss_df['model_name'].append(model_name)
        ll_loss_df['ll_loss'].append(logloss_value)
        ll_loss_df['micro_auc'].append(auc_value)
        ll_loss_df['accuracy'].append(  metrics.accuracy_score(all_predictions[model_name][fold_name]['y_val'], np.argmax(all_predictions[model_name][fold_name]['y_pred_val'], axis=1 ) )   )
        macro_auc = np.mean([metrics.roc_auc_score(all_predictions[model_name][fold_name]['y_val'] == i, all_predictions[model_name][fold_name]['y_pred_val'][:,i]) for i in range(2)])
        ll_loss_df['macro_auc'].append(macro_auc)
        temp1 = pd.DataFrame(all_predictions[model_name][fold_name]['y_val'], columns=[f'y_true'])
        temp1['fold'] = fold_name
        temp2 = pd.DataFrame(all_predictions[model_name][fold_name]['y_pred_val'], columns=[f'{model_name}-y_pred{i}' for i in range(all_predictions[model_name][fold_name]['y_pred_val'].shape[1])])
        data_list.append(pd.concat([temp1, temp2], axis=1).set_index(['fold', 'y_true']))
    full_dataframe_list.append(pd.concat(data_list, axis=0))
    

In [5]:
def f_mean(x):
    return str(round(np.mean(list(x)), 4)) + '+/-' + str(np.std(list(x)).round(2))
    
full_dataframe = pd.concat(full_dataframe_list, axis=1)
print ('-'*100)
print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


L = full_dataframe.copy()

----------------------------------------------------------------------------------------------------
                                      ll_loss      micro_auc       accuracy  \
model_name                                                                    
ad_imaging#LogisticRegression    0.329+/-0.02  0.9365+/-0.01  0.8882+/-0.01   
ad_imaging#XGBClassifier         0.3931+/-0.0  0.9261+/-0.01  0.8659+/-0.01   
ad_imaging#skorchMLPClassifier  0.2918+/-0.01  0.9371+/-0.01  0.8858+/-0.01   

                                    macro_auc  
model_name                                     
ad_imaging#LogisticRegression   0.9365+/-0.01  
ad_imaging#XGBClassifier        0.9261+/-0.01  
ad_imaging#skorchMLPClassifier  0.9371+/-0.01  


/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_77605/1576132248.py:6: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


In [6]:

L.index = obs.index
# L.reset_index().to_csv("embeddings/all_concat_full_dataframe.csv", index=False)

def get_training_data_numpy(fold_name):
    full_dataframe_folds = full_dataframe.reset_index()
    full_dataframe_val = full_dataframe_folds[full_dataframe_folds['fold']==fold_name]
    full_dataframe_train = full_dataframe_folds[~(full_dataframe_folds['fold']==fold_name)]
    X_train = full_dataframe_train.iloc[:, 2:].values.astype(np.float32)
    X_val = full_dataframe_val.iloc[:, 2:].values.astype(np.float32)
    y_train = full_dataframe_train.iloc[:, 1].values.astype(np.int64)
    y_val = full_dataframe_val.iloc[:, 1].values.astype(np.int64)
    return X_train, X_val, y_train, y_val


In [7]:

from omegaconf import OmegaConf
from sklearn.linear_model import LogisticRegression
# from lightgbm import LGBMClassifier
# from PredictiveModeling.src_basic.skorch_models import skorchMLPClassifier

# optuna_config_grid = OmegaConf.load(f"{ROOT_DIR_INSIDE}/PredictiveModeling/src_basic/{config['tuning_type']}_hyperparameters_with_index.json")

L = full_dataframe.copy()
L.index = obs.index

In [8]:
import pickle
with open(RESULTS_DIR / f'predictiveAnalysis/execute_ensemble_models/ensemble_data/{fname}_ensembleWeights_best_model_prediction.pkl', 'rb') as t:
    ensemble_weights = pickle.load(t)

In [9]:
ensemble_weights

{'ad_imaging#LogisticRegression': 0.4403008303258688,
 'ad_imaging#skorchMLPClassifier': 0.5000166735791762,
 'ad_imaging#XGBClassifier': 0.05968249609495491}

In [10]:
d_ensemble = {}
for col in full_dataframe.columns:
    w = ensemble_weights[col.split('-')[0]]
    pred_name = col.split('-')[1]
    d_ensemble['ensemble-'+pred_name] = d_ensemble.get('ensemble-'+pred_name, 0) + w * full_dataframe[col].values
    

ensembled_prediction = pd.DataFrame(d_ensemble)
ensembled_prediction.index = full_dataframe.index
ensembled_prediction = ensembled_prediction.reset_index()
from collections import defaultdict
def f_mean(x):
    return str(round(np.mean(list(x)), 4)) + '+/-' + str(np.std(list(x)).round(2))
    
g = list(ensembled_prediction.groupby('fold'))
cv_all_results = defaultdict(list)

for i in range(len(g)):  
    temp = g[i][1]
    cv_all_results['const'].append(0)
    cv_all_results['fold'].append(g[i][0])
    macro_auc = np.mean([metrics.roc_auc_score(temp['y_true'].values == i, temp.iloc[:, 2:].values[:,i]) for i in range(2)])
    cv_all_results['cv_macro_auc'].append(macro_auc)
    cv_all_results['cv_logloss'].append(  metrics.log_loss(temp['y_true'].values, temp.iloc[:, 2:])   )
    cv_all_results['cv_micro_auc'].append(  metrics.roc_auc_score(temp['y_true'].values, temp.iloc[:, 3] , multi_class='ovo' ) )
    cv_all_results['cv_accuracy'].append(  metrics.accuracy_score(temp['y_true'].values, np.argmax(temp.iloc[:, 2:].values, axis=1 ) )   )
    

avg = pd.DataFrame(cv_all_results).groupby('const').agg(f_mean).reset_index()
avg['fold'] = 'fold'


cv_all_results_df = pd.DataFrame(cv_all_results)
print ('-'*100)
print (pd.concat([cv_all_results_df, avg]))


print ('-'*100)
print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


----------------------------------------------------------------------------------------------------
   const   fold   cv_macro_auc     cv_logloss   cv_micro_auc   cv_accuracy
0      0  fold1       0.936627       0.272516       0.936627      0.908467
1      0  fold2        0.95157       0.259985        0.95157      0.902331
2      0  fold3       0.944026       0.287015       0.944026      0.892252
0      0   fold  0.9441+/-0.01  0.2732+/-0.01  0.9441+/-0.01  0.901+/-0.01
----------------------------------------------------------------------------------------------------
                                      ll_loss      micro_auc       accuracy  \
model_name                                                                    
ad_imaging#LogisticRegression    0.329+/-0.02  0.9365+/-0.01  0.8882+/-0.01   
ad_imaging#XGBClassifier         0.3931+/-0.0  0.9261+/-0.01  0.8659+/-0.01   
ad_imaging#skorchMLPClassifier  0.2918+/-0.01  0.9371+/-0.01  0.8858+/-0.01   

                           

/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_77605/4155623755.py:29: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  avg = pd.DataFrame(cv_all_results).groupby('const').agg(f_mean).reset_index()
/var/folders/y5/tr1yp82s2ns2_4g_6flxk45jrqb07t/T/ipykernel_77605/4155623755.py:39: FutureWarning: ['fold'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  print (pd.DataFrame(ll_loss_df).groupby('model_name').agg(f_mean))


In [11]:
len(cv_all_results)

6

In [12]:
cv_all_results

defaultdict(list,
            {'const': [0, 0, 0],
             'fold': ['fold1', 'fold2', 'fold3'],
             'cv_macro_auc': [0.9366271986941361,
              0.9515699945606488,
              0.944026472700028],
             'cv_logloss': [0.2725164836547759,
              0.25998528108327446,
              0.2870153780439914],
             'cv_micro_auc': [0.9366271986941361,
              0.9515699945606488,
              0.9440264727000279],
             'cv_accuracy': [0.9084668192219679,
              0.9023314429741651,
              0.8922518159806295]})

In [13]:
ensembled_prediction

,fold,y_true,ensemble-y_pred0,ensemble-y_pred1
0,fold1,1.0,0.484408,0.515592
1,fold1,0.0,0.904632,0.095368
2,fold1,0.0,0.926955,0.073045
3,fold1,0.0,0.974439,0.025561
4,fold1,0.0,0.828497,0.171503
...,...,...,...,...
4982,fold3,0.0,0.024210,0.975790
4983,fold3,0.0,0.894337,0.105663
4984,fold3,1.0,0.046581,0.953419
4985,fold3,0.0,0.941407,0.058593


In [14]:
ensembled_prediction.to_csv("data/ad_ensembled_prediction.csv", index=False)

In [15]:
print ("Number of subject", obs[obs['PHENO']=='Control'].index.map(lambda x: '-'.join(x.split('-')[:2])))
print ("Number of unique subject", obs[obs['PHENO']=='Control'].index.map(lambda x: '-'.join(x.split('-')[:2])).unique())

Number of subject Index(['ADNI-130_S_0969', 'ADNI-141_S_0726', 'ADNI-129_S_4369',
       'ADNI-141_S_0767', 'ADNI-037_S_6187', 'ADNI-072_S_0315',
       'ADNI-020_S_5203', 'ADNI-114_S_0166', 'ADNI-141_S_0767',
       'ADNI-141_S_0767',
       ...
       'ADNI-057_S_0643', 'ADNI-033_S_0923', 'ADNI-126_S_0605',
       'ADNI-006_S_0681', 'ADNI-033_S_0741', 'ADNI-098_S_0896',
       'ADNI-098_S_0896', 'ADNI-098_S_0896', 'ADNI-127_S_0260',
       'ADNI-126_S_0605'],
      dtype='object', name='ID', length=3161)
Number of unique subject Index(['ADNI-130_S_0969', 'ADNI-141_S_0726', 'ADNI-129_S_4369',
       'ADNI-141_S_0767', 'ADNI-037_S_6187', 'ADNI-072_S_0315',
       'ADNI-020_S_5203', 'ADNI-114_S_0166', 'ADNI-070_S_6966',
       'ADNI-099_S_0533',
       ...
       'ADNI-057_S_0643', 'ADNI-128_S_0188', 'ADNI-019_S_4835',
       'ADNI-133_S_0433', 'ADNI-131_S_0436', 'ADNI-128_S_0863',
       'ADNI-131_S_0319', 'ADNI-098_S_0896', 'ADNI-052_S_1168',
       'ADNI-006_S_0681'],
      dtype='ob

In [16]:
print ("Number of subject", obs[obs['PHENO']=='Dementia'].index.map(lambda x: '-'.join(x.split('-')[:2])))
print ("Number of unique subject", obs[obs['PHENO']=='Dementia'].index.map(lambda x: '-'.join(x.split('-')[:2])).unique())

Number of subject Index(['ADNI-126_S_4891', 'ADNI-018_S_0286', 'ADNI-018_S_0286',
       'ADNI-018_S_0286', 'ADNI-036_S_1240', 'ADNI-126_S_4891',
       'ADNI-067_S_0812', 'ADNI-126_S_4891', 'ADNI-018_S_0286',
       'ADNI-135_S_4954',
       ...
       'ADNI-133_S_1055', 'ADNI-014_S_0563', 'ADNI-041_S_1010',
       'ADNI-014_S_0328', 'ADNI-027_S_0179', 'ADNI-098_S_0269',
       'ADNI-136_S_0695', 'ADNI-011_S_0241', 'ADNI-126_S_0891',
       'ADNI-011_S_0241'],
      dtype='object', name='ID', length=1826)
Number of unique subject Index(['ADNI-126_S_4891', 'ADNI-018_S_0286', 'ADNI-036_S_1240',
       'ADNI-067_S_0812', 'ADNI-135_S_4954', 'ADNI-137_S_0796',
       'ADNI-099_S_0111', 'ADNI-036_S_0759', 'ADNI-137_S_0366',
       'ADNI-023_S_0084',
       ...
       'ADNI-098_S_0269', 'ADNI-041_S_1425', 'ADNI-014_S_0356',
       'ADNI-041_S_1010', 'ADNI-131_S_0497', 'ADNI-041_S_1412',
       'ADNI-011_S_0241', 'ADNI-098_S_0667', 'ADNI-014_S_0328',
       'ADNI-027_S_0179'],
      dtype='ob

In [17]:
obs[obs['PHENO']=='Control'].index.map(lambda x: '-'.join(x.split('-')[:2]))

Index(['ADNI-130_S_0969', 'ADNI-141_S_0726', 'ADNI-129_S_4369',
       'ADNI-141_S_0767', 'ADNI-037_S_6187', 'ADNI-072_S_0315',
       'ADNI-020_S_5203', 'ADNI-114_S_0166', 'ADNI-141_S_0767',
       'ADNI-141_S_0767',
       ...
       'ADNI-057_S_0643', 'ADNI-033_S_0923', 'ADNI-126_S_0605',
       'ADNI-006_S_0681', 'ADNI-033_S_0741', 'ADNI-098_S_0896',
       'ADNI-098_S_0896', 'ADNI-098_S_0896', 'ADNI-127_S_0260',
       'ADNI-126_S_0605'],
      dtype='object', name='ID', length=3161)

In [18]:
obs = pd.concat([get_obs_data_anndata('fold1'), get_obs_data_anndata('fold2'), get_obs_data_anndata('fold3')])